# Cloning and installing YOLOv5

In [1]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 10732, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10732 (delta 3), reused 6 (delta 1), pack-reused 10722
Receiving objects: 100% (10732/10732), 10.89 MiB | 15.86 MiB/s, done.
Resolving deltas: 100% (7408/7408), done.


In [2]:
!pip install -r yolov5/requirements.txt

# Write YAML file with directories

In [3]:
%%writefile ./yolov5/data/GBR.yaml

path: /kaggle/working/datasets/train_images
train: video_0
val: video_1
test:

nc: 1
names: [ 'starfish' ]

Writing ./yolov5/data/GBR.yaml


# Copy images to working directory

In [4]:
!mkdir datasets
!cp -r ../input/tensorflow-great-barrier-reef/train_images ./datasets

# Prepare labels

In [5]:
import pandas as pd
import ast

df = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv')
df = df[df['annotations'] != '[]']

for index, row in df.iterrows():
    label_list = []
    
    for d in ast.literal_eval(row['annotations']):
        label_list.append([0, (d.get('x') + (d.get('width') / 2)) / 1280, 
                           (d.get('y') + (d.get('height') / 2)) / 720, 
                           d.get('width') / 1280, d.get('height') / 720])
    
    label_df = pd.DataFrame(label_list)
    label_df.to_csv('./datasets/train_images/video_' + str(row['video_id']) + '/' + str(row['image_id'][2:]) + '.txt', 
                    header = False, index = False, sep = ' ')

# Train model

In [6]:
!python ./yolov5/train.py --img 640 --batch 16 --epochs 1 --data GBR.yaml --weights yolov5n.pt

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5n.pt, cfg=, data=GBR.yaml, hyp=yolov5/data/hyps/hyp.scratch.yaml, epochs=1, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-216-g856d4e5 torch 1.9.1 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epoch